## Tratamento dos dados
---
Inicialmente, temos os dados "crus", retirados do IBGE. Estes dados mostram o valor monetário da produção de cada um dos setores, e em que parte da cadeia produtiva ele se encontra.

In [2]:
import re

import pandas as pd
import numpy as np

In [3]:
# link da tabela:
# https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9085-matriz-de-insumo-produto.html?=&t=resultados
matriz = pd.read_excel("Matriz_de_Insumo_Produto_2015_Nivel_67.xls",
                       "03", skiprows=3)
matriz

,Unnamed: 0,Unnamed: 1,Valor da produção,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não ferrosos, inclusive beneficiamentos",...,9700\nServiços domésticos,Total\ndo produto,Exportação\nde bens e\nserviços,Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01911,"Arroz, trigo e outros cereais",11036.0,166.0,164.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8843.0,1280.0,0.0,0.0,450.0,0.0,463.0,2193.0,11036.0
2,01912,Milho em grão,29975.0,566.0,2314.0,9.0,0.0,0.0,0.0,0.0,...,0.0,11179.0,14537.0,0.0,0.0,3869.0,0.0,390.0,18796.0,29975.0
3,01913,"Algodão herbáceo, outras fibras da lavoura tem...",8943.0,260.0,23.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3722.0,4477.0,0.0,0.0,6.0,0.0,738.0,5221.0,8943.0
4,01914,Cana-de-açúcar,46080.0,615.0,148.0,4.0,0.0,0.0,0.0,0.0,...,0.0,44764.0,0.0,0.0,0.0,1316.0,0.0,0.0,1316.0,46080.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,97001,Serviços domésticos,61996.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,61996.0,0.0,0.0,61996.0,61996.0
128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,Total,NaN,10226869.0,115679.0,55512.0,6546.0,8674.0,71054.0,23165.0,9023.0,...,0.0,4092532.0,767032.0,1181278.0,87323.0,3185733.0,934157.0,-21186.0,6134337.0,10226869.0
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Problemas a serem resolvidos: 

- Colunas com texto e valores ``NaN``;

- A maior parte das colunas tem números e um '\n' no nome;

- Os setores estão subdivididos (não é uma matriz quadrada);

Primeiro, apagamos as linhas desnecessárias

In [4]:
matriz.drop(index=[0, 128, 130, 131], inplace=True)
matriz

,Unnamed: 0,Unnamed: 1,Valor da produção,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não ferrosos, inclusive beneficiamentos",...,9700\nServiços domésticos,Total\ndo produto,Exportação\nde bens e\nserviços,Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
1,01911,"Arroz, trigo e outros cereais",11036.0,166.0,164.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8843.0,1280.0,0.0,0.0,450.0,0.0,463.0,2193.0,11036.0
2,01912,Milho em grão,29975.0,566.0,2314.0,9.0,0.0,0.0,0.0,0.0,...,0.0,11179.0,14537.0,0.0,0.0,3869.0,0.0,390.0,18796.0,29975.0
3,01913,"Algodão herbáceo, outras fibras da lavoura tem...",8943.0,260.0,23.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3722.0,4477.0,0.0,0.0,6.0,0.0,738.0,5221.0,8943.0
4,01914,Cana-de-açúcar,46080.0,615.0,148.0,4.0,0.0,0.0,0.0,0.0,...,0.0,44764.0,0.0,0.0,0.0,1316.0,0.0,0.0,1316.0,46080.0
5,01915,Soja em grão,109170.0,1864.0,155.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39597.0,68968.0,0.0,0.0,86.0,0.0,519.0,69573.0,109170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,94801,"Organizações patronais, sindicais e outros ser...",83334.0,104.0,29.0,14.0,35.0,458.0,178.0,52.0,...,0.0,5502.0,0.0,0.0,44348.0,33484.0,0.0,0.0,77832.0,83334.0
125,94802,"Manutenção de computadores, telefones e objeto...",26429.0,0.0,0.0,5.0,0.0,73.0,0.0,2.0,...,0.0,16898.0,0.0,0.0,0.0,9531.0,0.0,0.0,9531.0,26429.0
126,94803,Serviços pessoais,42366.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3702.0,0.0,0.0,0.0,38664.0,0.0,0.0,38664.0,42366.0
127,97001,Serviços domésticos,61996.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,61996.0,0.0,0.0,61996.0,61996.0


In [48]:
colunas = list(matriz.columns)

matriz_insumo_produto = pd.DataFrame(columns=matriz.columns)

for i in range(len(colunas)):
    """
    Se verificar, irá ver que "0191 Agricultura, inclusive o apoio à agricultura e o pós-colheita"
    está dividido em 10 partes, todas com um código começando com 0191!
    Isso permite a automatização da soma das subdivisões setoriais.
    """
    indices = []
    
    for j in range(1, len(matriz.index)):
        
        subsetor = matriz["Unnamed: 0"][j]
        
        if subsetor[:4] in list(matriz.columns)[i]:
            # se o começo do código bate com o código desta coluna, marque-a para ser adicionada
            indices.append(matriz.index[j-1])
            
    if len(indices) != 0:
        # se há colunas para serem somadas, as some e as adicione na matriz_insumo_produto
        matriz_insumo_produto = matriz_insumo_produto.append(
            matriz.loc[indices[0]:indices[-1]].sum(),
            ignore_index=True
        )
        
matriz_insumo_produto

,Unnamed: 0,Unnamed: 1,Valor da produção,"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita","0192\nPecuária, inclusive o apoio à pecuária",0280\nProdução florestal; pesca e aquicultura,0580\nExtração de carvão mineral e de minerais não metálicos,"0680\nExtração de petróleo e gás, inclusive as atividades de apoio","0791\nExtração de minério de ferro, inclusive beneficiamentos e a aglomeração","0792\nExtração de minerais metálicos não ferrosos, inclusive beneficiamentos",...,9700\nServiços domésticos,Total\ndo produto,Exportação\nde bens e\nserviços,Consumo\ndo governo,Consumo\ndas\n ISFLSF,Consumo \ndas famílias,Formação bruta\nde capital fixo,Variação\nde estoque,Demanda\nfinal,Demanda\ntotal
0,019110191201913019140191501916019170191801919,"Arroz, trigo e outros cereaisMilho em grãoAlgo...",302321.0,6657.0,3635.0,78.0,0.0,0.0,0.0,0.0,...,0.0,136659.0,111125.0,36.0,0.0,58210.0,305.0,-4014.0,165662.0,302321.0
1,01921019220192301924,"Bovinos e outros animais vivos, produtos anima...",128920.0,521.0,4475.0,45.0,0.0,0.0,0.0,0.0,...,0.0,95810.0,1997.0,2.0,0.0,18007.0,12206.0,898.0,33110.0,128920.0
2,0280102802,Produtos da exploração florestal e da silvicul...,34101.0,915.0,1095.0,1728.0,2.0,0.0,0.0,0.0,...,0.0,15217.0,2294.0,4.0,0.0,16466.0,752.0,-632.0,18884.0,34101.0
3,0580105802,Carvão mineralMinerais não metálicos,20365.0,46.0,356.0,11.0,324.0,534.0,0.0,1.0,...,0.0,17986.0,1896.0,0.0,0.0,0.0,0.0,483.0,2379.0,20365.0
4,06801,"Petróleo, gás natural e serviços de apoio",162130.0,0.0,0.0,0.0,1.0,12079.0,46.0,14.0,...,0.0,111409.0,38789.0,0.0,0.0,0.0,8548.0,3384.0,50721.0,162130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,86911,Saúde pública,175278.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,175278.0,0.0,0.0,0.0,0.0,175278.0,175278.0
63,86921,Saúde privada,224955.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,21182.0,227.0,35091.0,7583.0,160872.0,0.0,0.0,203773.0,224955.0
64,90801,"Serviços de artes, cultura, esporte e recreação",33796.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5966.0,1753.0,0.0,7742.0,18335.0,0.0,0.0,27830.0,33796.0
65,948019480294803,"Organizações patronais, sindicais e outros ser...",152129.0,104.0,29.0,19.0,35.0,531.0,178.0,54.0,...,0.0,26102.0,0.0,0.0,44348.0,81679.0,0.0,0.0,126027.0,152129.0


Agora corrigimos os nomes das colunas

In [49]:
colunas = list(matriz_insumo_produto.columns)
for i in range(len(colunas)):
    colunas[i] = re.sub('\d', "", colunas[i])  # Se há algum número no nome da coluna, o substitua por nada ("")
    colunas[i] = re.sub('\n', " ", colunas[i])  # Se há algum "\n" (pular para nova linha) o substituia por um espaço (" ")
    if colunas[i][0] == " ":
        colunas[i] = colunas[i][1::]  # Se o primeiro caracter por um espaço, retire-o
    if colunas[i][-1] == " ":
        colunas[i] = colunas[i][:-1]  # Mesmo raciocínio, mas para o último caracter
        
matriz_insumo_produto.columns=colunas
matriz_insumo_produto

,Unnamed:,Unnamed:,Valor da produção,"Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não metálicos,"Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos",...,Serviços domésticos,Total do produto,Exportação de bens e serviços,Consumo do governo,Consumo das ISFLSF,Consumo das famílias,Formação bruta de capital fixo,Variação de estoque,Demanda final,Demanda total
0,019110191201913019140191501916019170191801919,"Arroz, trigo e outros cereaisMilho em grãoAlgo...",302321.0,6657.0,3635.0,78.0,0.0,0.0,0.0,0.0,...,0.0,136659.0,111125.0,36.0,0.0,58210.0,305.0,-4014.0,165662.0,302321.0
1,01921019220192301924,"Bovinos e outros animais vivos, produtos anima...",128920.0,521.0,4475.0,45.0,0.0,0.0,0.0,0.0,...,0.0,95810.0,1997.0,2.0,0.0,18007.0,12206.0,898.0,33110.0,128920.0
2,0280102802,Produtos da exploração florestal e da silvicul...,34101.0,915.0,1095.0,1728.0,2.0,0.0,0.0,0.0,...,0.0,15217.0,2294.0,4.0,0.0,16466.0,752.0,-632.0,18884.0,34101.0
3,0580105802,Carvão mineralMinerais não metálicos,20365.0,46.0,356.0,11.0,324.0,534.0,0.0,1.0,...,0.0,17986.0,1896.0,0.0,0.0,0.0,0.0,483.0,2379.0,20365.0
4,06801,"Petróleo, gás natural e serviços de apoio",162130.0,0.0,0.0,0.0,1.0,12079.0,46.0,14.0,...,0.0,111409.0,38789.0,0.0,0.0,0.0,8548.0,3384.0,50721.0,162130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,86911,Saúde pública,175278.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,175278.0,0.0,0.0,0.0,0.0,175278.0,175278.0
63,86921,Saúde privada,224955.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,21182.0,227.0,35091.0,7583.0,160872.0,0.0,0.0,203773.0,224955.0
64,90801,"Serviços de artes, cultura, esporte e recreação",33796.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5966.0,1753.0,0.0,7742.0,18335.0,0.0,0.0,27830.0,33796.0
65,948019480294803,"Organizações patronais, sindicais e outros ser...",152129.0,104.0,29.0,19.0,35.0,531.0,178.0,54.0,...,0.0,26102.0,0.0,0.0,44348.0,81679.0,0.0,0.0,126027.0,152129.0


VBP: Valor Bruto da Produção

DF: Demanda Final

In [50]:
# em caixa alta pois são valores constantes e não variáveis

VBP = matriz_insumo_produto["Demanda total"]  
DF = matriz_insumo_produto["Demanda final"]

Retiramos agora as três primeiras colunas

``inplace=True`` aplica o método .drop() na própria matriz

In [8]:
matriz_insumo_produto.drop(columns=colunas[:3], axis=1, inplace=True)
matriz_insumo_produto.drop(columns=colunas[70:], inplace=True)

In [16]:
matriz_2 = matriz_insumo_produto.drop(columns=colunas[69]).drop(index=66).copy()

Resolvido! Agora temos nossa matriz de insumo-produto pronta para ser manipulada!

In [17]:
import pymrio

Agora, importamos um pacote específico para análises de matriz de Insumo-Produto: o *pymrio*.

*Pymrio: Multi-Regional Input-Output Analysis in Python - A python module for automating io calculations and generating reports*

In [37]:
matriz_A = pymrio.calc_A(matriz_insumo_produto, VBP)
matriz_A

,"Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não metálicos,"Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos","Abate e produtos de carne, inclusive os produtos do laticínio e da pesca",Fabricação e refino de açúcar,Outros produtos alimentares,...,Outras atividades administrativas e serviços complementares,"Atividades de vigilância, segurança e investigação","Administração pública, defesa e seguridade social",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetáculos",Organizações associativas e outros serviços pessoais,Serviços domésticos
0,0.022020,0.028196,0.002287,0.000000,0.000000,0.000000,0.000000,0.003495,0.704846,0.205766,...,0.000586,0.000000,0.001005,0.001397,0.000450,0.001044,0.000902,0.000030,0.001459,0.0
1,0.001723,0.034711,0.001320,0.000000,0.000000,0.000000,0.000000,0.335512,0.000000,0.005761,...,0.000000,0.000000,0.000177,0.000305,0.000088,0.000126,0.000485,0.000000,0.000053,0.0
2,0.003027,0.008494,0.050673,0.000098,0.000000,0.000000,0.000000,0.005400,0.000000,0.000769,...,0.000000,0.000000,0.000091,0.000225,0.000035,0.000165,0.000058,0.000000,0.000000,0.0
3,0.000152,0.002761,0.000323,0.015910,0.003294,0.000000,0.000060,0.000196,0.000280,0.000957,...,0.000000,0.000000,0.000080,0.000059,0.000000,0.000017,0.000004,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000049,0.074502,0.000874,0.000844,0.000092,0.000000,0.000870,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
63,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000089,0.000017,0.000000,0.000074,0.093814,0.000000,0.000000,0.0
64,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000101,0.000000,0.001046,0.000288,0.000000,0.000382,0.000000,0.023879,0.008992,0.0
65,0.000344,0.000225,0.000557,0.001719,0.003275,0.003381,0.003254,0.000703,0.001763,0.000722,...,0.005402,0.000929,0.001050,0.000392,0.004714,0.008695,0.009246,0.000503,0.002629,0.0


In [38]:
matriz_L = pymrio.calc_L(matriz_A)
matriz_L

,"Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária",Produção florestal; pesca e aquicultura,Extração de carvão mineral e de minerais não metálicos,"Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos","Abate e produtos de carne, inclusive os produtos do laticínio e da pesca",Fabricação e refino de açúcar,Outros produtos alimentares,...,Outras atividades administrativas e serviços complementares,"Atividades de vigilância, segurança e investigação","Administração pública, defesa e seguridade social",Educação pública,Educação privada,Saúde pública,Saúde privada,"Atividades artísticas, criativas e de espetáculos",Organizações associativas e outros serviços pessoais,Serviços domésticos
0,1.027523,0.056085,0.010155,0.004839,0.003528,0.003526,0.005037,0.051567,0.753179,0.250904,...,0.001776,0.001305,0.003226,0.004201,0.001674,0.005422,0.004193,0.001289,0.007103,0.0
1,0.002005,1.041045,0.001761,0.000162,0.000200,0.000162,0.000211,0.381400,0.001591,0.010069,...,0.000339,0.000106,0.001795,0.002771,0.000984,0.002876,0.002516,0.000263,0.002911,0.0
2,0.004008,0.010536,1.053728,0.000587,0.000423,0.000494,0.000604,0.011494,0.003216,0.003528,...,0.000573,0.000128,0.000435,0.000625,0.000355,0.000711,0.000853,0.000304,0.000758,0.0
3,0.002909,0.004904,0.000822,1.017199,0.004548,0.000898,0.001348,0.002581,0.002951,0.002648,...,0.000629,0.000102,0.000730,0.000348,0.000239,0.000741,0.000373,0.000379,0.000570,0.0
4,0.023635,0.020266,0.010180,0.028393,1.093400,0.031566,0.045212,0.021755,0.039870,0.023487,...,0.003917,0.004455,0.003507,0.002744,0.003516,0.003760,0.003461,0.004052,0.007939,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0
63,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000098,0.000019,0.000000,0.000082,1.103527,0.000000,0.000000,0.0
64,0.000145,0.000270,0.000138,0.000239,0.000301,0.000234,0.000247,0.000515,0.000376,0.000553,...,0.000452,0.000211,0.001380,0.000444,0.000700,0.000756,0.000260,1.025823,0.009709,0.0
65,0.001689,0.001780,0.001250,0.003421,0.005260,0.005513,0.005708,0.003172,0.004953,0.003184,...,0.006599,0.001730,0.002641,0.001204,0.005988,0.010056,0.011354,0.002063,1.004495,0.0


Chegamos enfim à matriz de coeficientes!

Agora podemos exportar estas matrizes para arquivos csv

In [52]:
import os

pasta = "Matrizes"

if not os.path.exists(pasta):
    # garante que a pasta onde quero salvar estes arquivos existe
    os.mkdir(pasta)

matriz_insumo_produto.to_csv("Matrizes\matriz_insumo_produto.csv")
matriz_A.to_csv("Matrizes\matriz_A.csv")
matriz_L.to_excel("Matrizes\matriz_coeficientes.csv")
VBP.to_csv("Matrizes\VBP.csv", header=True)
DF.to_csv("Matrizes\demanda_final.csv", header=True)

## Análise dos dados

---

Com a matriz de coeficientes pronta, podemos realizar as análises e descobrir o tipo dos setores

Os tipos dependem do coeficiente das ligações para frente e para trás. Calculamos isso da seguinte forma:

$$L_f = \frac{1/n\sum_{j}{b_{ij}}}{1/n²\sum_i\sum_j b_{ij}}$$


$$L_t = \frac{1/n\sum_{i}{b_{ij}}}{1/n²\sum_i\sum_j b_{ij}}$$

Ou seja: o coeficiente de ligação para frente da linha $i$ é igual à média dos coeficientes desta linha dividida pela média de todos os coeficientes da matriz;

E o coeficiente de ligações para trás da coluna $j$ é igual à média dos coeficientes desta coluna dividida pela média total dos coeficientes.

Podemos dividir os setores em quatro tipos:

- **Tipo I:** ($L_t < 1; L_f < 1$) Geralmente independente, sem ligações ou ligações fracas;

- **Tipo II:** ($L_t < 1; L_f > 1$) Depende da demanda interisdustrial, mais relações à frente;

- **Tipo III:** ($L_t > 1; L_f > 1$) Geralmente dependente, é um ***setor chave***;

- **Tipo IV:** ($L_t > 1; L_f < 1$) Depende da oferta interindustrial, mais relações para trás;

In [178]:
# é preciso usar .to_numpy().sum() para somar todos os valores do DataFrame de forma eficiente
media = matriz_L.to_numpy().sum() / matriz_L.shape[0]**2 

l_f = matriz_L.mean() / media 
l_t = matriz_L.mean(axis=1) / media # axis=1 para média das colunas

Series([], dtype: float64)

In [175]:
matriz_L.mean(axis=1) / media

0     1.862446
1     0.870680
2     0.762387
3     0.688005
4     1.406660
        ...   
62    0.553941
63    0.611399
64    0.640074
65    0.724536
66    0.553941
Length: 67, dtype: float64

In [163]:
?pd.read_csv